# Introduction
Customer churn prediction is a critical task for businesses aiming to retain customers and reduce revenue loss. This notebook demonstrates how to build and evaluate multiple machine learning models to predict customer churn using Python's pandas and scikit-learn libraries.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set seaborn style for better visuals
sns.set(style="whitegrid")


### Loading the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('churn_data.csv')

# Display the first few rows
df.head()

### Exploratory Data Analysis (EDA)

In [ ]:
# Check the shape of the dataset
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

# Check for missing values
missing_values = df.isnull().sum()
print("\nMissing Values in Each Column:\n", missing_values)

# Summary statistics
df.describe()

# Distribution of the target variable
sns.countplot(x='Churn', data=df)
plt.title('Distribution of Churn')
plt.show()

# Visualize distribution of numerical features
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numerical_features].hist(bins=30, figsize=(15, 5))
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()


### Data Preprocessing

In [ ]:
# Drop 'customerID' as it does not contribute to churn prediction
df.drop('customerID', axis=1, inplace=True)

# Handle missing values in 'TotalCharges'
# Convert 'TotalCharges' to numeric, coerce errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check missing values after conversion
print("\nMissing Values After Conversion:\n", df.isnull().sum())

# Fill missing 'TotalCharges' with median
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Verify no missing values remain
print("\nMissing Values After Imputation:\n", df.isnull().sum())

# Encode 'gender' column
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])  # Female=0, Male=1

# Encode binary categorical variables
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

# Encode 'MultipleLines' - considering 'No phone service' as separate category
df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 'No'})
df['MultipleLines'] = df['MultipleLines'].map({'Yes': 1, 'No': 0})

# Encode 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
# 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod'

# List of categorical columns to encode
categorical_cols = ['InternetService', 'OnlineSecurity', 'OnlineBackup',
                    'DeviceProtection', 'TechSupport', 'StreamingTV',
                    'StreamingMovies', 'Contract', 'PaymentMethod']

# One-Hot Encoding for categorical variables
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Feature Scaling for numerical features
scaler = StandardScaler()
df[['tenure', 'MonthlyCharges', 'TotalCharges']] = scaler.fit_transform(df[['tenure', 'MonthlyCharges', 'TotalCharges']])

# Display the preprocessed data
df.head()


### Feature Engineering

In [ ]:
# Create 'FamilySize' feature by combining 'Dependents' and 'Partner'
df['FamilySize'] = df['Dependents'] + df['Partner']

# Drop 'Dependents' and 'Partner' if 'FamilySize' is sufficient
df.drop(['Dependents', 'Partner'], axis=1, inplace=True)

# Create 'HasMultipleServices' feature
service_cols = ['PhoneService', 'MultipleLines', 'InternetService_Fiber optic',
               'InternetService_No', 'OnlineSecurity_Yes', 'OnlineBackup_Yes',
               'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes',
               'StreamingMovies_Yes']
df['HasMultipleServices'] = df[service_cols].sum(axis=1)

# Drop individual service columns if 'HasMultipleServices' is sufficient
df.drop(service_cols, axis=1, inplace=True)

# Display the updated DataFrame
df.head()


### Splitting the Data

In [ ]:
# Define features and target variable
X = df.drop('Churn', axis=1)
y = df['Churn']

# Split the data with stratification to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Display the shape of the splits
print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")


### Model Training and Evaluation

In [ ]:
# Initialize Logistic Regression
logreg = LogisticRegression(random_state=42)

# Train the model
logreg.fit(X_train, y_train)

# Predict on test data
y_pred_logreg = logreg.predict(X_test)

# Evaluate the model
print("Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_logreg))
print("\nClassification Report:\n", classification_report(y_test, y_pred_logreg))


In [ ]:
# Initialize Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on test data
y_pred_rf = rf.predict(X_test)

# Evaluate the model
print("Random Forest Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


In [ ]:
# Initialize Support Vector Machine
svm = SVC(random_state=42, probability=True)

# Train the model
svm.fit(X_train, y_train)

# Predict on test data
y_pred_svm = svm.predict(X_test)

# Evaluate the model
print("Support Vector Machine")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC-AUC:", roc_auc_score(y_test, svm.predict_proba(X_test)[:,1]))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm))


In [ ]:
# Initialize K-Nearest Neighbors
knn = KNeighborsClassifier()

# Train the model
knn.fit(X_train, y_train)

# Predict on test data
y_pred_knn = knn.predict(X_test)

# Evaluate the model
print("K-Nearest Neighbors")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("ROC-AUC:", roc_auc_score(y_test, knn.predict_proba(X_test)[:,1]))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))


In [ ]:
# Initialize Gradient Boosting Classifier
gb = GradientBoostingClassifier(random_state=42)

# Train the model
gb.fit(X_train, y_train)

# Predict on test data
y_pred_gb = gb.predict(X_test)

# Evaluate the model
print("Gradient Boosting Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("ROC-AUC:", roc_auc_score(y_test, gb.predict_proba(X_test)[:,1]))
print("\nClassification Report:\n", classification_report(y_test, y_pred_gb))


### Comparing Models

In [ ]:
# Create a DataFrame to store evaluation metrics
models = ['Logistic Regression', 'Random Forest', 'SVM', 'KNN', 'Gradient Boosting']
accuracy = [
    accuracy_score(y_test, y_pred_logreg),
    accuracy_score(y_test, y_pred_rf),
    accuracy_score(y_test, y_pred_svm),
    accuracy_score(y_test, y_pred_knn),
    accuracy_score(y_test, y_pred_gb)
]
roc_auc = [
    roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]),
    roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]),
    roc_auc_score(y_test, svm.predict_proba(X_test)[:,1]),
    roc_auc_score(y_test, knn.predict_proba(X_test)[:,1]),
    roc_auc_score(y_test, gb.predict_proba(X_test)[:,1])
]

comparison_df = pd.DataFrame({
    'Model': models,
    'Accuracy': accuracy,
    'ROC-AUC': roc_auc
})

# Display the comparison
comparison_df


In [ ]:
## Visualization

# Set the figure size
plt.figure(figsize=(10,6))

# Plot Accuracy
sns.barplot(x='Model', y='Accuracy', data=comparison_df, palette='viridis')
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.show()

# Plot ROC-AUC
plt.figure(figsize=(10,6))
sns.barplot(x='Model', y='ROC-AUC', data=comparison_df, palette='magma')
plt.title('Model ROC-AUC Comparison')
plt.ylabel('ROC-AUC Score')
plt.ylim(0,1)
plt.show()


### Hyperparameter Tuning

In [ ]:
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize GridSearchCV
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='roc_auc'
)

# Fit GridSearchCV
grid_search_rf.fit(X_train, y_train)

# Best parameters
print("Best Parameters for Random Forest:", grid_search_rf.best_params_)

# Best estimator
best_rf = grid_search_rf.best_estimator_

# Predict with the best estimator
y_pred_best_rf = best_rf.predict(X_test)

# Evaluate the tuned model
print("Tuned Random Forest Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_best_rf))
print("ROC-AUC:", roc_auc_score(y_test, best_rf.predict_proba(X_test)[:,1]))
print("\nClassification Report:\n", classification_report(y_test, y_pred_best_rf))


### Feature Importance

In [ ]:
# Feature importance from Random Forest
importances = best_rf.feature_importances_
feature_names = X.columns
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)

# Display top 10 important features
top_features = feature_importances.head(10)
top_features

# Plot feature importances
plt.figure(figsize=(10,6))
sns.barplot(x=top_features, y=top_features.index, palette='coolwarm')
plt.title('Top 10 Feature Importances (Random Forest)')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()
